In [ ]:
get_ipython().run_line_magic('pip', 'install db-dtypes')

In [ ]:
get_ipython().system('pip install keras')
get_ipython().system('pip install tensorflow')

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt  
import statsmodels.api as sm
import itertools

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'

In [ ]:
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data` 
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data` 
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
wagedata = query_job.to_dataframe()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list` 
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
prediction_list = query_job.to_dataframe()

In [ ]:
import pandas as pd
import numpy as np
combined = pd.merge(data,wagedata,on=['uu_id','countyfips', 'tract','tract_name'],how = 'left')

In [ ]:
wagedata[wagedata['uu_id']=='a5c6dcff737e183f7931b472f10c3235']

In [ ]:
combined['average_wage'].fillna(combined['average_wage'].mean(),inplace = True)
## Using the mean for the missing wage

In [ ]:
firstgroup = ['edu_8th_or_less','edu_grades_9_11','edu_hs_grad_equiv','edu_post_hs','edu_unknown']
secondgroup = ['gender_female','gender_male','gender_na']
thirdgroup = ['race_amerindian','race_asian','race_black','race_noanswer','race_hawaiiannative','race_other','race_white']
## columns to impute

In [ ]:
complete = combined.dropna(axis = 0)
#using the complete rows to estimate the ratio of each catagory occupying the claims
prob = {}
for i in firstgroup:
    prob[i] = (complete[i]/complete['total_claims']).mean()
for i in secondgroup:
    prob[i] = (complete[i]/complete['total_claims']).mean()
for i in thirdgroup:
    prob[i] = (complete[i]/complete['total_claims']).mean()

In [ ]:
prediction_list['total_claims'] = 0

In [ ]:
prediction_list

In [ ]:
prediction_list.loc[:,['uu_id','total_claims','week_number']].to_csv('submission_prediction_output(2).csv',index = False)

In [ ]:
approximatedimputation(combined)
combined.to_csv('submission_prediction_output(1).csv',index = False)

In [ ]:
def approximatedimputation(data):
    firstgroup = ['edu_8th_or_less','edu_grades_9_11','edu_hs_grad_equiv','edu_post_hs','edu_unknown']
    secondgroup = ['gender_female','gender_male','gender_na']
    thirdgroup = ['race_amerindian','race_asian','race_black','race_noanswer','race_hawaiiannative','race_other','race_white']
    for i in firstgroup:
        for j in range(data.shape[0]):
            # if is nan, we use the approximation method to try to impute
            if pd.isna(data[i].values[j]):
                data[i].values[j] = int(prob[i]*data['total_claims'].values[j])
    for i in secondgroup:
        for j in range(data.shape[0]):
            if pd.isna(data[i].values[j]):
                data[i].values[j] = int(prob[i]*data['total_claims'].values[j])
    for i in thirdgroup:
        for j in range(data.shape[0]):
            if pd.isna(data[i].values[j]):
                data[i].values[j] = int(prob[i]*data['total_claims'].values[j])

In [ ]:
approximatedimputation(combined)
combined.to_csv('submission_prediction_output(1).csv',index = False)